In [1]:
from DataParser import DataParser
import numpy as np
import scipy.sparse as sps
from sklearn.model_selection import KFold

from Data_manager.split_functions.split_train_validation_random_holdout import \
split_train_in_two_percentage_global_sample

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from SimpleEvaluator import evaluator
from Base.Evaluation.Evaluator import EvaluatorHoldout


In [2]:
from Base.NonPersonalizedRecommender import TopPop, Random
from Hybrid.HybridCombinationSearchCV import HybridCombinationMergedSearchCV, HybridCombinationSearchCV2
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.SpecialItemKNNCBFRecommender import SpecialItemKNNCBFRecommender
from MatrixFactorization.IALSRecommender import IALSRecommender
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from SLIM_ElasticNet.SSLIM_ElasticNet import SSLIMElasticNet
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender
from MatrixFactorization.NMFRecommender import NMFRecommender
from KNN.ItemKNN_CBF_CF import ItemKNN_CBF_CF

In [3]:
seed = 1666
parser = DataParser()

URM_all = parser.get_URM_all()
ICM_all = parser.get_ICM_all()

#URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.85, seed=seed)

**single rec here**

In [4]:
rp3b = (RP3betaRecommender, {'topK': 926, 'alpha': 0.4300109351916609, 'beta': 0.01807360750913967, 'normalize_similarity': False})
p3a = (P3alphaRecommender, {'topK': 575, 'alpha': 0.48009885897470206, 'normalize_similarity': False})
icf = (ItemKNNCFRecommender, {'topK': 1000, 'shrink': 1000, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'TF-IDF'})
ucf = (UserKNNCFRecommender, {'topK': 163, 'shrink': 846, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'TF-IDF'})
icb = (ItemKNNCBFRecommender, {'topK': 164, 'shrink': 8, 'similarity': 'jaccard', 'normalize': True})
sbpr = (SLIM_BPR_Cython, {'topK': 1000, 'epochs': 130, 'symmetric': False, 'sgd_mode': 'adam', 'lambda_i': 1e-05, 'lambda_j': 1e-05, 'learning_rate': 0.0001})
icfcb = (ItemKNN_CBF_CF, {'topK': 1000, 'shrink': 1000, 'similarity': 'asymmetric', 'normalize': True, 'asymmetric_alpha': 0.241892724784089, 'feature_weighting': 'TF-IDF', 'icm_weight': 1.0})
sslim = (SSLIMElasticNet, {'beta': 0.567288665094892, 'topK': 1000, 'l1_ratio': 1e-05, 'alpha': 0.001})
psvd = (PureSVDRecommender, {'num_factors': 350})
ials = (IALSRecommender, {})
icbsup = (SpecialItemKNNCBFRecommender, {'topK': 1000, 'shrink': 1000, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'BM25'})

In [5]:
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=seed)

URM_list = []
URM_test_list = []
evaluator_list = []

shape = URM_all.shape
indptr = URM_all.indptr
indices = URM_all.indices
data = URM_all.data

for train_index, test_index in kf.split(data):
    data_train = np.ones(data.shape)
    data_test = np.ones(data.shape)
    data_train[test_index] = 0
    data_test[train_index] = 0
    kf_train = sps.csr_matrix((data_train, indices, indptr), shape=shape).copy()
    kf_test = sps.csr_matrix((data_test, indices, indptr), shape=shape).copy()
    kf_train.eliminate_zeros()
    kf_test.eliminate_zeros()
    URM_list.append(kf_train)
    URM_test_list.append(kf_test)
    evaluator_list.append(EvaluatorHoldout(kf_test, cutoff_list=[10]))

In [6]:
#0: label, 1: type, 2: list, 3:params
recommenders = [
    #single
    ('icb', 'single', icb, {}),
    ('icbsup', 'single', icbsup, {}),
    ('rp3b', 'single', rp3b, {}),
    ('p3a', 'single', p3a, {}),
    ('icf', 'single', icf, {}),
    ('ucf', 'single', ucf, {}),
    ('sbpr', 'single', sbpr, {}),
    ('sslim', 'single', sslim, {}),
    ('psvd', 'single', psvd, {}),
    ('ials', 'single', ials, {}),
    ('icfcb', 'single', icfcb, {}),
    #linear
    ('icbsup, icfcb, p3a', 'linear', [icbsup, icfcb, p3a], {'a': 0.993188337424128, 'b': 0.9909175139221994, 'c': 0.7815444492201006, 'normalize': False}),
('icbsup, icfcb, rp3b', 'linear', [icbsup, icfcb, rp3b], {'a': 0.2901802313422713, 'b': 0.4392120921456557, 'c': 0.16625489563936016, 'normalize': False}),
('icb, icfcb, p3a', 'linear', [icb, icfcb, p3a], {'a': 0.39988908366603393, 'b': 0.7467067794473331, 'c': 0.2068190996377532, 'normalize': False}),
('icbsup, ucf, p3a', 'linear', [icbsup, ucf, p3a], {'a': 0.9662016322698685, 'b': 0.4963111377096092, 'c': 0.6874259637686088, 'normalize': False}),
('icbsup, p3a, sslim', 'linear', [icbsup, p3a, sslim], {'a': 0.11340363717192684, 'b': 0.07221362174970904, 'c': 0.7644867129062151, 'normalize': False}),
('icbsup, rp3b, sbpr', 'linear', [icbsup, rp3b, sbpr], {'a': 0.6351784979513733, 'b': 0.4227964938023694, 'c': 0.96815170415434, 'normalize': False}),
('icbsup, p3a, sbpr', 'linear', [icbsup, p3a, sbpr], {'a': 0.8495173762195323, 'b': 0.7916707351347642, 'c': 0.0, 'normalize': False}),
('icbsup, sbpr, sslim', 'linear', [icbsup, sbpr, sslim], {'a': 1.0, 'b': 0.548724319649282, 'c': 1.0, 'normalize': True}),
('icb, icfcb, sbpr', 'linear', [icb, icfcb, sbpr], {'a': 0.36471733011095414, 'b': 1.0, 'c': 1.0, 'normalize': False}),
('icbsup, rp3b, sslim', 'linear', [icbsup, rp3b, sslim], {'a': 0.9943636298152313, 'b': 0.6338212977896964, 'c': 0.9064091710184868, 'normalize': False}),
('icbsup, icfcb, sbpr', 'linear', [icbsup, icfcb, sbpr], {'a': 0.16721122903353033, 'b': 0.6078522485010392, 'c': 1.0, 'normalize': False}),
('icb, icfcb, ucf', 'linear', [icb, icfcb, ucf], {'a': 0.2127784681579103, 'b': 0.60649347517527, 'c': 0.1328508812429561, 'normalize': False}),
('icbsup, icf, rp3b', 'linear', [icbsup, icf, rp3b], {'a': 0.974238032123889, 'b': 0.05681121439278215, 'c': 0.7588862294834752, 'normalize': False}),
('icbsup, icf, p3a', 'linear', [icbsup, icf, p3a], {'a': 1.0, 'b': 0.0, 'c': 1.0, 'normalize': False}),
('icbsup, ucf, rp3b', 'linear', [icbsup, ucf, rp3b], {'a': 0.8978309114164305, 'b': 0.4854872045282991, 'c': 0.5843535501206276, 'normalize': False}),
('icbsup, rp3b, p3a', 'linear', [icbsup, rp3b, p3a], {'a': 0.8490806214717617, 'b': 0.22519798480808623, 'c': 0.5961181908448233, 'normalize': False}),
('icbsup, icfcb, ucf', 'linear', [icbsup, icfcb, ucf], {'a': 0.5406791705129667, 'b': 1.0, 'c': 1.0, 'normalize': False}),
('icb, icfcb, rp3b', 'linear', [icb, icfcb, rp3b], {'a': 0.39988908366603393, 'b': 0.7467067794473331, 'c': 0.2068190996377532, 'normalize': False}),
('icbsup, ucf, sbpr', 'linear', [icbsup, ucf, sbpr], {'a': 0.24668897504169815, 'b': 0.46103517802781613, 'c': 1.0, 'normalize': False}),
('icb, icfcb, sslim', 'linear', [icb, icfcb, sslim], {'a': 0.18900439671024385, 'b': 0.7278340234785393, 'c': 0.492470586021196, 'normalize': False}),
('icbsup, ucf, sslim', 'linear', [icbsup, ucf, sslim], {'a': 0.19083263978347026, 'b': 0.36822743561222177, 'c': 0.6433901864235739, 'normalize': False}),
('icb, icfcb, icf', 'linear', [icb, icfcb, icf], {'a': 0.2127784681579103, 'b': 0.60649347517527, 'c': 0.1328508812429561, 'normalize': False}),
('icbsup, icf, ucf', 'linear', [icbsup, icf, ucf], {'a': 0.7142854575392344, 'b': 1.0, 'c': 1.0, 'normalize': False}),
('icbsup, icf, sbpr', 'linear', [icbsup, icf, sbpr], {'a': 0.2155408847729527, 'b': 0.5365195928500357, 'c': 1.0, 'normalize': False}),
('icbsup, icfcb, icf', 'linear', [icbsup, icfcb, icf], {'a': 0.29991875042603844, 'b': 1.0, 'c': 0.472604045914412, 'normalize': False}),
('icb, sbpr, sslim', 'linear', [icb, sbpr, sslim], {'a': 0.5460201378519737, 'b': 0.32024341642443616, 'c': 0.9020078710965056, 'normalize': True}),
('icb, p3a, sslim', 'linear', [icb, p3a, sslim], {'a': 0.4342307597489174, 'b': 0.25354973464932506, 'c': 1.0, 'normalize': False}),
('icbsup, icfcb, sslim', 'linear', [icbsup, icfcb, sslim], {'a': 1.0, 'b': 1.0, 'c': 1.0, 'normalize': True}),
('icbsup, icf, sslim', 'linear', [icbsup, icf, sslim], {'a': 0.8349204523211081, 'b': 0.4597288428914985, 'c': 1.0, 'normalize': True}),
('icb, ucf, p3a', 'linear', [icb, ucf, p3a], {'a': 0.8978309114164305, 'b': 0.4854872045282991, 'c': 0.5843535501206276, 'normalize': False}),
('icb, rp3b, sslim', 'linear', [icb, rp3b, sslim], {'a': 0.44111403803963356, 'b': 0.20563284056244352, 'c': 0.5755894297091031, 'normalize': False}),
('icb, ucf, sslim', 'linear', [icb, ucf, sslim], {'a': 0.5983518768664847, 'b': 1.0, 'c': 0.6034427806571803, 'normalize': True}),
('icb, rp3b, sbpr', 'linear', [icb, rp3b, sbpr], {'a': 0.6600574451473287, 'b': 0.3831354491973874, 'c': 1.0, 'normalize': False}),
('icb, ucf, rp3b', 'linear', [icb, ucf, rp3b], {'a': 0.42497238811530047, 'b': 0.06513381063211422, 'c': 0.24394909361657563, 'normalize': False}),
('icb, icf, rp3b', 'linear', [icb, icf, rp3b], {'a': 0.42497238811530047, 'b': 0.06513381063211422, 'c': 0.24394909361657563, 'normalize': False}),
('icb, icf, p3a', 'linear', [icb, icf, p3a], {'a': 0.8978309114164305, 'b': 0.4854872045282991, 'c': 0.5843535501206276, 'normalize': False}),
('icb, rp3b, p3a', 'linear', [icb, rp3b, p3a], {'a': 0.42497238811530047, 'b': 0.06513381063211422, 'c': 0.24394909361657563, 'normalize': False}),
('icb, p3a, sbpr', 'linear', [icb, p3a, sbpr], {'a': 0.9781575812965135, 'b': 0.6416852376253805, 'c': 0.8119823355130084, 'normalize': False}),
('icb, icf, sslim', 'linear', [icb, icf, sslim], {'a': 0.5800285287224096, 'b': 0.28976905492467897, 'c': 1.0, 'normalize': True}),
('icb, ucf, sbpr', 'linear', [icb, ucf, sbpr], {'a': 0.19083263978347026, 'b': 0.36822743561222177, 'c': 0.6433901864235739, 'normalize': False}),
    #merged
    ('icbsup, icfcb, p3a', 'merged', [icbsup, icfcb, p3a], {'alpha': 0.581116263967674, 'l1_ratio': 0.5598903069887389, 'topK': 1000, 'normalize': True}),
('icb, icfcb, sbpr', 'merged', [icb, icfcb, sbpr], {'alpha': 0.6829163194586242, 'l1_ratio': 0.386516642359119, 'topK': 1000, 'normalize': True}),
('icbsup, icfcb, rp3b', 'merged', [icbsup, icfcb, rp3b], {'alpha': 0.7269115930024891, 'l1_ratio': 0.6051868432813969, 'topK': 807, 'normalize': False}),
('icb, icfcb, p3a', 'merged', [icb, icfcb, p3a], {'alpha': 0.7097149976708894, 'l1_ratio': 0.4253431147789698, 'topK': 683, 'normalize': True}),
('icbsup, p3a, sslim', 'merged', [icbsup, p3a, sslim], {'alpha': 0.6509731450181614, 'l1_ratio': 0.5325849264151943, 'topK': 736, 'normalize': True}),
('icbsup, rp3b, sslim', 'merged', [icbsup, rp3b, sslim], {'alpha': 0.16451383666897593, 'l1_ratio': 0.7280753668239933, 'topK': 622, 'normalize': False}),
('icbsup, sbpr, sslim', 'merged', [icbsup, sbpr, sslim], {'alpha': 0.535827782580078, 'l1_ratio': 0.42823755538774455, 'topK': 228, 'normalize': True}),
('icbsup, p3a, sbpr', 'merged', [icbsup, p3a, sbpr], {'alpha': 0.48791974774529073, 'l1_ratio': 0.5476114683349933, 'topK': 749, 'normalize': False}),
('icbsup, p3a, rp3b', 'merged', [icbsup, p3a, rp3b], {'alpha': 1.0, 'l1_ratio': 0.3749359476595637, 'topK': 1000, 'normalize': True}),
('icbsup, icf, rp3b', 'merged', [icbsup, icf, rp3b], {'alpha': 0.5086347515387531, 'l1_ratio': 0.8318853016648462, 'topK': 1000, 'normalize': True}),
('icbsup, rp3b, sbpr', 'merged', [icbsup, rp3b, sbpr], {'alpha': 0.7702171365467229, 'l1_ratio': 0.4846943139192311, 'topK': 985, 'normalize': True}),
('icbsup, icf, p3a', 'merged', [icbsup, icf, p3a], {'alpha': 0.5334947443510734, 'l1_ratio': 0.7670738264844964, 'topK': 984, 'normalize': True}),
('icb, sbpr, sslim', 'merged', [icb, sbpr, sslim], {'alpha': 0.5191303017757435, 'l1_ratio': 0.5397979031860276, 'topK': 1000, 'normalize': True}),
('icbsup, icf, sbpr', 'merged', [icbsup, icf, sbpr], {'alpha': 0.2956440034792403, 'l1_ratio': 0.29893177702984014, 'topK': 1000, 'normalize': False}),
('icb, icfcb, rp3b', 'merged', [icb, icfcb, rp3b], {'alpha': 0.5837741870266273, 'l1_ratio': 0.537759713449255, 'topK': 840, 'normalize': True}),
('icb, icfcb, sslim', 'merged', [icb, icfcb, sslim], {'alpha': 0.5349175218474181, 'l1_ratio': 0.2576251556087315, 'topK': 1000, 'normalize': False}),
('icbsup, icfcb, sbpr', 'merged', [icbsup, icfcb, sbpr], {'alpha': 0.42057870266300357, 'l1_ratio': 0.09026142584874454, 'topK': 274, 'normalize': False}),
('icb, p3a, sslim', 'merged', [icb, p3a, sslim], {'alpha': 0.5337201740049957, 'l1_ratio': 0.6238228001711947, 'topK': 1000, 'normalize': True}),
('icb, icfcb, icf', 'merged', [icb, icfcb, icf], {'alpha': 0.8564169787002618, 'l1_ratio': 0.38051833534630997, 'topK': 349, 'normalize': True}),
('icb, rp3b, sslim', 'merged', [icb, rp3b, sslim], {'alpha': 0.5214551035450364, 'l1_ratio': 0.6435085766039979, 'topK': 1000, 'normalize': True}),
('icbsup, icfcb, icf', 'merged', [icbsup, icfcb, icf], {'alpha': 0.8911893260873708, 'l1_ratio': 0.190809944835244, 'topK': 678, 'normalize': False}),
('icbsup, icf, sslim', 'merged', [icbsup, icf, sslim], {'alpha': 0.17007254940564234, 'l1_ratio': 0.5199674749734902, 'topK': 268, 'normalize': False}),
('icb, rp3b, sbpr', 'merged', [icb, rp3b, sbpr], {'alpha': 0.6279733305260499, 'l1_ratio': 0.6112981510597016, 'topK': 1000, 'normalize': False}),
('icb, rp3b, p3a', 'merged', [icb, rp3b, p3a], {'alpha': 0.930884826476267, 'l1_ratio': 0.6534945725958449, 'topK': 772, 'normalize': False}),
('icb, p3a, rp3b', 'merged', [icb, p3a, rp3b], {'alpha': 0.5746538230434903, 'l1_ratio': 0.7233709611102526, 'topK': 904, 'normalize': True}),
('icbsup, icfcb, sslim', 'merged', [icbsup, icfcb, sslim], {'alpha': 0.8096670993132076, 'l1_ratio': 0.4281911898108999, 'topK': 600, 'normalize': True}),
('icb, icf, p3a', 'merged', [icb, icf, p3a], {'alpha': 0.44923735666844766, 'l1_ratio': 0.9950962886988683, 'topK': 990, 'normalize': True}),
('icb, icf, sslim', 'merged', [icb, icf, sslim], {'alpha': 0.27657784163744875, 'l1_ratio': 0.9973921663552862, 'topK': 908, 'normalize': True}),
('icb, p3a, sbpr', 'merged', [icb, p3a, sbpr], {'alpha': 0.7269115930024891, 'l1_ratio': 0.6051868432813969, 'topK': 807, 'normalize': False}),
]

In [7]:
num_rec = len(recommenders)

In [ ]:
f_ranges = [
    (0,4),
    (4,7),
    (7,13),
    (13,26),
    (26,-1)
]

MAP_per_group = []
cutoff = 10

for f_range in f_ranges:
    
    result_dict={}
    for k in range (num_rec):
        r = []
        for i in range (len(URM_list)):
            URM_test_group = parser.filter_URM_test_by_range(URM_list[i], URM_test_list[i], f_range)
            evaluator_test = EvaluatorHoldout(URM_test_group, cutoff_list=[cutoff])
            
            if recommenders[k][1] == 'single':
                rec = recommenders[k][2][0]
                try:
                    rec = rec(URM_list[i], ICM_all, verbose = False)
                except:
                    rec = rec(URM_list[i], verbose=False)
                try:
                    rec.load_model(f'stored_recommenders/seed_{str(seed)}_hybrid_search/{rec.RECOMMENDER_NAME}/', f'{str(seed)}_fold-{str(i)}')
                    print(f"{rec.RECOMMENDER_NAME} loaded. [seed={seed}, fold={i}]")
                except:
                    print(f"Fitting {rec.RECOMMENDER_NAME} ... [seed={seed}, fold={i}]")
                    rec.fit(**recommenders[k][2][1])
                    print(f"done.")
                    rec.save_model(f'stored_recommenders/seed_{str(seed)}_hybrid_search/{rec.RECOMMENDER_NAME}/', f'{str(seed)}_fold-{str(i)}')
            elif recommenders[k][1] == 'linear':
                rec = HybridCombinationSearchCV2(URM_list[i], ICM_all, recommenders[k][2], seed = seed, fold = i)
                rec.fit(**recommenders[k][3])
            elif recommenders[k][1] == 'merged':
                rec = HybridCombinationMergedSearchCV(URM_list[i], ICM_all, recommenders[k][2], seed = seed, fold = i)
                rec.fit(**recommenders[k][3])
            else:
                raise NameError

            results, _ = evaluator_test.evaluateRecommender(rec)
            r.append(results[cutoff]["MAP"])
        result_dict[k]=np.average(r)

    MAP_per_group.append(result_dict)

ItemKNNCBFRecommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 1687 ( 100.00% ) in 0.96 sec. Users per second: 1766
ItemKNNCBFRecommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 1640 ( 100.00% ) in 0.84 sec. Users per second: 1946
ItemKNNCBFRecommender loaded. [seed=1666, fold=2]
EvaluatorHoldout: Processed 1700 ( 100.00% ) in 0.89 sec. Users per second: 1913
ItemKNNCBFRecommender loaded. [seed=1666, fold=3]
EvaluatorHoldout: Processed 1640 ( 100.00% ) in 0.87 sec. Users per second: 1886
ItemKNNCBFRecommender loaded. [seed=1666, fold=4]
EvaluatorHoldout: Processed 1633 ( 100.00% ) in 0.82 sec. Users per second: 1982
Special-ItemKNNCBFRec loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 1687 ( 100.00% ) in 0.94 sec. Users per second: 1795
Special-ItemKNNCBFRec loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 1640 ( 100.00% ) in 0.94 sec. Users per second: 1749
Special-ItemKNNCBFRec loaded. [seed=1666, fold=2]
EvaluatorHoldout: Processed 1700 

S-SLIMElasticNet loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 1687 ( 100.00% ) in 1.27 sec. Users per second: 1326
S-SLIMElasticNet loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 1640 ( 100.00% ) in 1.26 sec. Users per second: 1302
S-SLIMElasticNet loaded. [seed=1666, fold=2]
EvaluatorHoldout: Processed 1700 ( 100.00% ) in 1.30 sec. Users per second: 1312
S-SLIMElasticNet loaded. [seed=1666, fold=3]
EvaluatorHoldout: Processed 1640 ( 100.00% ) in 1.27 sec. Users per second: 1289
S-SLIMElasticNet loaded. [seed=1666, fold=4]
EvaluatorHoldout: Processed 1633 ( 100.00% ) in 1.25 sec. Users per second: 1309
PureSVDRecommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 1687 ( 100.00% ) in 1.21 sec. Users per second: 1399
PureSVDRecommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 1640 ( 100.00% ) in 1.37 sec. Users per second: 1198
PureSVDRecommender loaded. [seed=1666, fold=2]
EvaluatorHoldout: Processed 1700 ( 100.00% ) in 1.47 sec. Users per

HybridCombinationSearchCV: URM Detected 65 (0.82 %) cold users.
HybridCombinationSearchCV: URM Detected 2338 (9.00 %) cold items.
Special-ItemKNNCBFRec loaded. [seed=1666, fold=3]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=3]
RP3betaRecommender loaded. [seed=1666, fold=3]
EvaluatorHoldout: Processed 1640 ( 100.00% ) in 1.45 sec. Users per second: 1134
HybridCombinationSearchCV: URM Detected 77 (0.97 %) cold users.
HybridCombinationSearchCV: URM Detected 2398 (9.23 %) cold items.
Special-ItemKNNCBFRec loaded. [seed=1666, fold=4]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=4]
RP3betaRecommender loaded. [seed=1666, fold=4]
EvaluatorHoldout: Processed 1633 ( 100.00% ) in 1.46 sec. Users per second: 1122
HybridCombinationSearchCV: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCV: URM Detected 2381 (9.17 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=0]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=0]
P3alphaRecommender loaded. [seed=1666, 

In [ ]:
to_be_plotted = []
for i in range(0, len (f_ranges)):
    print(f"RANGE {f_ranges[i]}")
    maxp = 0
    old_maxp = 0
    difference = 0
    old_name = ''
    max_name=''
    max_key=None
    old_key=None
    
    result_dict = MAP_per_group[i]
    for key, value in result_dict.items():
        #value = combo_dict.get(key)
        print(f"> MAP {recommenders[key][1]}-{recommenders[key][0]}:\t {value}")
        if value > maxp:
            difference=value - maxp
            old_maxp=maxp
            maxp=value
            old_name=max_name
            max_name=recommenders[key][0]
            old_key=max_key
            max_key=key
        elif value > old_maxp:
            difference=maxp - value
            old_maxp=value
            old_name=recommenders[key][0]
            old_key=key
           
    if max_key not in to_be_plotted:
        to_be_plotted.append(max_key)
    
    if old_key not in to_be_plotted:
        to_be_plotted.append(old_key)
    
    print(f"Max MAP is {maxp} from {max_name}({recommenders[max_key][1]}) with difference from previous max of {difference} from {old_name}({recommenders[old_key][1]}) \n")
        

In [ ]:
import matplotlib.pyplot as pyplot
%matplotlib inline  
pyplot.figure(figsize=(20,15))
for key in to_be_plotted:
    l=[]
    for result_dict in MAP_per_group:
        l.append(result_dict[key])
    if np.random.random() > 0.5:
        style = '--o'
    else:
        style = '--x'
    label = recommenders[key][0]
    """
    else:
        for merged_dict in MAP_merged_per_group:
            l.append(merged_dict[key])
        style = '--x'
        label = merged_labels[key]
        """
    pyplot.plot(l, style, label = label)

pyplot.xticks(ticks=range(0,len(f_ranges)), labels=f_ranges)
pyplot.ylabel('MAP')
pyplot.xlabel('User Group')
pyplot.legend()
pyplot.show()